In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import magic_input_processing as magic

C:\data\mobius\PythonWrapper\MAGIC\magic_input_processing.py:184: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if sea_salt_ref is 'Cl' :
C:\data\mobius\PythonWrapper\MAGIC\magic_input_processing.py:228: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sea_salt_ref is not 'Cl' :
C:\data\mobius\PythonWrapper\MAGIC\magic_input_processing.py:236: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (ca_ref_range is not None) and element is 'Ca' :


In [2]:
# Make an empty dataframe that will later contain your input data.
# The range provided is the years (inclusive) that you want in your input data.
df = magic.empty_input_df(1850, 2020)

# Use the pandas library to read in data from various locations. You need them in a pandas dataframe form.
# You can e.g. use pd.read_excel or pd.read_csv depending on your source.

# Right now precipitation and air temperature has to be daily values (I will maybe make it accept a source that is monthly later)
# The columns can be called whatever you want.

met_df = pd.read_excel('example_met_Birkenes.xls', sheet_name='Ark1', index_col='Date', parse_dates=True)

met_df

FileNotFoundError: [Errno 2] No such file or directory: 'example_met_Birkenes.xls'

In [ ]:
# Example of a df that can be used to patch the deposition series. The values are relative to a reference year that you provide
# yourself below. You can also read this in from an excel file like above. The one here is made for Birkenes

emep_df = magic.get_emep_deposition_scales()

emep_df

In [3]:
# Similarly, read in runoff data from excel. This does not have to be in a separate document from the precip and air temp.
# That is just how I organized this example.

runoff_df = pd.read_excel('example_runoff_Birkenes.xls', sheet_name='Ark1', index_col='Date', parse_dates=True)

runoff_df

FileNotFoundError: [Errno 2] No such file or directory: 'example_runoff_Birkenes.xls'

In [4]:
# Read in deposition data (concentration in precipitation). These can be daily, monthly or yearly in the source.
# The columns should include SO4, NO3, NH4, Ca, and one of Mg or Cl (depending on what you use for sea salt correction)

dep_df = pd.read_excel('example_dep_Birkenes.xls', sheet_name='Ark1', index_col='year', parse_dates=True)

dep_df

FileNotFoundError: [Errno 2] No such file or directory: 'example_dep_Birkenes.xls'

In [5]:
# Read in observed runoff concentrations. You should at least have SO4
# and one of Cl or Mg (depending on which one you use for sea salt correction).

runoff_chem_df = pd.read_excel('example_runoff_chem_Birkenes.xls', sheet_name='Ark1', index_col='year', parse_dates=True)

runoff_chem_df

FileNotFoundError: [Errno 2] No such file or directory: 'example_runoff_chem_Birkenes.xls'

In [6]:

# Add the precipitation time series to the input data. The patch ref range is the range that it uses for computing
# averages that it uses to patch in missing values. The series you provide has to be without holes from its own start to end.
magic.add_precip_ts(df, met_df['P'], patch_ref_range=(1981, 1991))

# Add a runoff series.
#NOTE: The Runoff series should not be patched at all. It will only be used for reference in the deposition correction
#      and for calibrating the eventual hydr. module.
magic.add_runoff_ts(df, runoff_df['Q'])

# Add in air temperature series similarly to precipitation.
magic.add_air_temperature_ts(df, met_df['T'], patch_ref_range=(1971, 1981))


# Add in sea salt corrected deposition.
# Before adding this you have to add Precipitation, Runoff series to your input dataframe.
# The runoff series has to be valid for the sea_salt_ref_range. The precipitation and measured deposition series should be
# valid for the patch_ref_range and the ca_ref_range. Note that measured deposition series are linearly interpolated between
# valid values, so it is ok with a few holes.

'''
df  : The dataframe you want to write the inputs to. It must already have a 'Precipitation' and a 'Runoff' column
dep_df : A date-indexed dataframe with observed concentrations in precipitation, with columns ['Ca', 'Mg', 'Na', 'K', 'NH4', 'SO4', 'Cl', 'NO3', 'F']   F is optional
runoff_chem_df : A date-indexed dataframe with observed concentrations in runoff, with the same columns as source_df.
source_is_mg_l : True if the unit of the source_df is mg/l, False if they are meq/m3=µeq/l
runoff_is_mg_l : True if the unit of the runoff_df is mg/l, False if they are meq/m3=µeq/l
n_dep_given_as_n : Only relevant if source is mg/l: True if the mass of NO3 and NH4 deposition is given as the mass of N, False if it is given as the mass of the full molecule.
s_dep_given_as_s : Only relevant if source is mg/l: True if the mass of SO4 deposition is given as the mass of S, False if it is given as the mass of the full molecule.
sea_salt_ref : The reference deposition series for sea salt correction. Usually 'Cl', but can also use 'Mg'.
sea_salt_ref_range : pair (start_year, end_year). This is the range that is used to compute the sea_salt_factor and the SO4* factor.
patch_scale : A date-indexed dataframe containing some of the columns of the input_df. Should have some values of historical deposition relative to a reference year. Example : get_emep_deposition_scales()
patch_ref_range : pair (start_year, end_year). Range of years to compute the value for the reference value that scales the values used for patching.
so4_weathering : how much of the so4 runoff should be accounted for by weathering and not deposition
ca_ref_range : If not None, compute Ca excess deposition relative to SO4 excess deposition with reference to this range.
ca_excess_weight : Ca excess deposition relative to SO4 excess deposition is multiplied with this value
n_dry_deposition_factor : Multiply NH4 and NO3 deposition by this value
'''

magic.add_deposition_ts(df, dep_df, runoff_chem_df, sea_salt_ref='Mg', sea_salt_ref_range=(1981,2000), 
                        source_is_mg_l=True, runoff_is_mg_l=False,
                        patch_scale=emep_df, patch_ref_range=(1989,1991),
                        so4_weathering=30.0,
                        ca_ref_range=(1974,2002), ca_excess_weight=0.5, 
                        n_dry_deposition_factor=1.14)

df

NameError: name 'met_df' is not defined

In [7]:
# Plot the series that you have added to your input dataframe

magic.plot_inputs(df)

ValueError: Number of rows must be a positive integer, not 0

<Figure size 432x288 with 0 Axes>

In [9]:
# Export the input dataframe to a Mobius input file.

magic.write_as_input_file(df, 'test_inputs.dat')